https://github.com/Suji04/NormalizedNerd/blob/master/Introduction%20to%20NLP/How%20to%20use%20pre-trained%20word%20vectors%20(GloVe%2BWord2Vec).ipynb

https://www.youtube.com/watch?v=oMd7sMlxYFk&ab_channel=NormalizedNerd

In [ ]:

#https://github.com/Suji04/NormalizedNerd/blob/master/Introduction%20to%20NLP/How%20to%20use%20pre-trained%20word%20vectors%20(GloVe%2BWord2Vec).ipynb
#https://www.youtube.com/watch?v=oMd7sMlxYFk&ab_channel=NormalizedNerd



In [ ]:

import numpy as np
import pandas as pd
import gensim
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
#https://github.com/ys7yoo/nea_tf/issues/2
#from keras.layers.embeddings import Embedding
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

from google.colab import drive
drive.mount("/GloVe")


Mounted at /GloVe


In [ ]:

data=pd.read_csv("/GloVe/MyDrive/NLP/Word Embeddings/GloVe /cleaned_hm.csv")
data.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,affection
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,affection
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,exercise
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,affection


In [ ]:

data['predicted_category'].value_counts()


affection           34168
achievement         33993
enjoy_the_moment    11144
bonding             10727
leisure              7458
nature               1843
exercise             1202
Name: predicted_category, dtype: int64

In [ ]:

data['num_sentence'].value_counts()

1     83711
2      9542
3      3847
4      1624
5       821
6       336
7       183
8       107
10       68
9        61
11       35
13       26
12       21
16       17
18       17
14       14
17       14
19       12
21       10
15        7
23        7
25        7
26        5
24        5
22        4
30        3
31        3
20        3
29        3
27        2
32        2
40        2
37        2
69        1
60        1
34        1
42        1
44        1
48        1
35        1
46        1
51        1
56        1
53        1
58        1
28        1
45        1
Name: num_sentence, dtype: int64

In [ ]:

# deleting happy moments with more than 10 sentences
mod_data=data.loc[data['num_sentence']<=10]
mod_data['predicted_category'].value_counts()

affection           34020
achievement         33966
enjoy_the_moment    11115
bonding             10700
leisure              7458
nature               1839
exercise             1202
Name: predicted_category, dtype: int64

In [ ]:
## categorical to numerical
encode = {
    "affection" : 0,
    "achievement"  : 1,       
    "bonding" : 2,    
    "enjoy_the_moment" : 3,     
    "leisure"  : 4,    
    "nature" : 5,    
    "exercise" : 6
}

In [ ]:
mod_data['predicted_category']= mod_data['predicted_category'].apply(lambda x: encode[x])
mod_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,27673,2053,24h,I went on a successful date with someone I fel...,I went on a successful date with someone I fel...,True,1,NaN,0
1,27674,2,24h,I was happy when my son got 90% marks in his e...,I was happy when my son got 90% marks in his e...,True,1,NaN,0
2,27675,1936,24h,I went to the gym this morning and did yoga.,I went to the gym this morning and did yoga.,True,1,NaN,6
3,27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,2
4,27677,6227,24h,I went with grandchildren to butterfly display...,I went with grandchildren to butterfly display...,True,1,NaN,0


**Data cleaning for NLP**

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

happy_lines = list()
lines = mod_data["cleaned_hm"].values.tolist()

In [ ]:
for line in lines:
    # tokenize the text
    tokens = word_tokenize(line)
    tokens = [w.lower() for w in tokens]
    # remove puntuations
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove non alphabetic characters
    words = [word for word in stripped if word.isalpha()]
    happy_lines.append(words)
    
happy_lines[0:5]

[['i',
  'went',
  'on',
  'a',
  'successful',
  'date',
  'with',
  'someone',
  'i',
  'felt',
  'sympathy',
  'and',
  'connection',
  'with'],
 ['i',
  'was',
  'happy',
  'when',
  'my',
  'son',
  'got',
  'marks',
  'in',
  'his',
  'examination'],
 ['i', 'went', 'to', 'the', 'gym', 'this', 'morning', 'and', 'did', 'yoga'],
 ['we',
  'had',
  'a',
  'serious',
  'talk',
  'with',
  'some',
  'friends',
  'of',
  'ours',
  'who',
  'have',
  'been',
  'flaky',
  'lately',
  'they',
  'understood',
  'and',
  'we',
  'had',
  'a',
  'good',
  'evening',
  'hanging',
  'out'],
 ['i',
  'went',
  'with',
  'grandchildren',
  'to',
  'butterfly',
  'display',
  'at',
  'crohn',
  'conservatory']]

**Train-test split**

In [ ]:
happy_lines

[['i',
  'went',
  'on',
  'a',
  'successful',
  'date',
  'with',
  'someone',
  'i',
  'felt',
  'sympathy',
  'and',
  'connection',
  'with'],
 ['i',
  'was',
  'happy',
  'when',
  'my',
  'son',
  'got',
  'marks',
  'in',
  'his',
  'examination'],
 ['i', 'went', 'to', 'the', 'gym', 'this', 'morning', 'and', 'did', 'yoga'],
 ['we',
  'had',
  'a',
  'serious',
  'talk',
  'with',
  'some',
  'friends',
  'of',
  'ours',
  'who',
  'have',
  'been',
  'flaky',
  'lately',
  'they',
  'understood',
  'and',
  'we',
  'had',
  'a',
  'good',
  'evening',
  'hanging',
  'out'],
 ['i',
  'went',
  'with',
  'grandchildren',
  'to',
  'butterfly',
  'display',
  'at',
  'crohn',
  'conservatory'],
 ['i', 'meditated', 'last', 'night'],
 ['i',
  'made',
  'a',
  'new',
  'recipe',
  'for',
  'peasant',
  'bread',
  'and',
  'it',
  'came',
  'out',
  'spectacular'],
 ['i',
  'got',
  'gift',
  'from',
  'my',
  'elder',
  'brother',
  'which',
  'was',
  'really',
  'surprising',
  'me

In [ ]:
validation_split = 0.2
#we only take 55 words of each review
max_length = 55
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(happy_lines)
happy_lines

[['i',
  'went',
  'on',
  'a',
  'successful',
  'date',
  'with',
  'someone',
  'i',
  'felt',
  'sympathy',
  'and',
  'connection',
  'with'],
 ['i',
  'was',
  'happy',
  'when',
  'my',
  'son',
  'got',
  'marks',
  'in',
  'his',
  'examination'],
 ['i', 'went', 'to', 'the', 'gym', 'this', 'morning', 'and', 'did', 'yoga'],
 ['we',
  'had',
  'a',
  'serious',
  'talk',
  'with',
  'some',
  'friends',
  'of',
  'ours',
  'who',
  'have',
  'been',
  'flaky',
  'lately',
  'they',
  'understood',
  'and',
  'we',
  'had',
  'a',
  'good',
  'evening',
  'hanging',
  'out'],
 ['i',
  'went',
  'with',
  'grandchildren',
  'to',
  'butterfly',
  'display',
  'at',
  'crohn',
  'conservatory'],
 ['i', 'meditated', 'last', 'night'],
 ['i',
  'made',
  'a',
  'new',
  'recipe',
  'for',
  'peasant',
  'bread',
  'and',
  'it',
  'came',
  'out',
  'spectacular'],
 ['i',
  'got',
  'gift',
  'from',
  'my',
  'elder',
  'brother',
  'which',
  'was',
  'really',
  'surprising',
  'me

In [ ]:

sequences = tokenizer_obj.texts_to_sequences(happy_lines)
sequences


[[1, 23, 16, 3, 716, 300, 13, 282, 1, 91, 9790, 5, 2148, 13],
 [1, 7, 15, 22, 2, 68, 17, 1016, 9, 66, 1563],
 [1, 23, 4, 6, 387, 34, 99, 5, 94, 998],
 [21,
  18,
  3,
  1745,
  318,
  13,
  40,
  50,
  11,
  3867,
  122,
  35,
  53,
  15158,
  1043,
  80,
  2501,
  5,
  21,
  18,
  3,
  41,
  308,
  953,
  29],
 [1, 23, 13, 1987, 4, 4299, 3868, 19, 15159, 15160],
 [1, 3528, 38, 72],
 [1, 20, 3, 27, 728, 8, 11621, 1297, 5, 14, 81, 29, 5874],
 [1, 17, 217, 28, 2, 2149, 169, 64, 7, 45, 2027, 10],
 [65, 2, 3019, 84, 32, 1, 140],
 [166, 3529, 2066, 13, 2, 241, 4815, 358],
 [1, 81, 9, 232, 9, 2, 329, 11, 2596, 210, 104],
 [1, 260, 2, 806, 374, 233, 522, 14, 195, 10, 105, 1700],
 [23, 4, 574, 13, 2, 50, 14, 7, 161],
 [1, 7, 15161, 1212, 5, 20, 28, 6, 1878],
 [737,
  969,
  14,
  145,
  61,
  1184,
  2686,
  4,
  92,
  28,
  2150,
  4,
  15,
  32,
  128,
  590,
  98,
  275,
  8,
  6,
  266,
  12,
  5542,
  128,
  29,
  11,
  291,
  2150,
  223,
  5,
  3869,
  128,
  9,
  6,
  2234,
  11,
  170

In [ ]:

# Doing the indexing of word
word_index = tokenizer_obj.word_index
word_index


{'i': 1,
 'my': 2,
 'a': 3,
 'to': 4,
 'and': 5,
 'the': 6,
 'was': 7,
 'for': 8,
 'in': 9,
 'me': 10,
 'of': 11,
 'that': 12,
 'with': 13,
 'it': 14,
 'happy': 15,
 'on': 16,
 'got': 17,
 'had': 18,
 'at': 19,
 'made': 20,
 'we': 21,
 'when': 22,
 'went': 23,
 'is': 24,
 'time': 25,
 'very': 26,
 'new': 27,
 'from': 28,
 'out': 29,
 'day': 30,
 'work': 31,
 'so': 32,
 'an': 33,
 'this': 34,
 'have': 35,
 's': 36,
 'her': 37,
 'last': 38,
 'friend': 39,
 'some': 40,
 'good': 41,
 'up': 42,
 'after': 43,
 'he': 44,
 'really': 45,
 'she': 46,
 'able': 47,
 'today': 48,
 'one': 49,
 'friends': 50,
 'all': 51,
 'family': 52,
 'been': 53,
 'first': 54,
 'about': 55,
 'nt': 56,
 'our': 57,
 'am': 58,
 'as': 59,
 'home': 60,
 'be': 61,
 'get': 62,
 'found': 63,
 'which': 64,
 'yesterday': 65,
 'his': 66,
 'but': 67,
 'son': 68,
 'dinner': 69,
 'great': 70,
 'not': 71,
 'night': 72,
 'finally': 73,
 'because': 74,
 'see': 75,
 'long': 76,
 'daughter': 77,
 'by': 78,
 'job': 79,
 'they': 80,
 '

In [ ]:

print("unique tokens - "+str(len(word_index)))


unique tokens - 26138


In [ ]:

vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab_size - '+str(vocab_size))


vocab_size - 26139


In [ ]:

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
lines_pad

array([[   1,   23,   16, ...,    0,    0,    0],
       [   1,    7,   15, ...,    0,    0,    0],
       [   1,   23,    4, ...,    0,    0,    0],
       ...,
       [1814,   13,    2, ...,    0,    0,    0],
       [   1,   18,    3, ...,    0,    0,    0],
       [   1,   18,    3, ...,    0,    0,    0]], dtype=int32)

In [ ]:

category =  mod_data['predicted_category'].values
category

array([0, 0, 6, ..., 0, 2, 6])

In [ ]:
indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
category = category[indices]

n_values = np.max(category) + 1
Y = np.eye(n_values)[category]

In [ ]:
num_validation_samples = int(validation_split * lines_pad.shape[0])

X_train_pad = lines_pad[:-num_validation_samples]
y_train = Y[:-num_validation_samples]
X_test_pad = lines_pad[-num_validation_samples:]
y_test = Y[-num_validation_samples:]

In [ ]:
print('Shape of X_train_pad:', X_train_pad.shape)
print('Shape of y_train:', y_train.shape)

print('Shape of X_test_pad:', X_test_pad.shape)
print('Shape of y_test:', y_test.shape)

Shape of X_train_pad: (80240, 55)
Shape of y_train: (80240, 7)
Shape of X_test_pad: (20060, 55)
Shape of y_test: (20060, 7)


In [28]:
#!unzip "/GloVe/MyDrive/NLP/Word Embeddings/GloVe /word2vec-GoogleNews-vectors-master.zip" -d "/GloVe/MyDrive/NLP/Word Embeddings/GloVe "

**GloVe Model**

In [26]:
embedding_index = {}
embedding_dim = 100
glove_dir = "/GloVe/MyDrive/glove.twitter.27B"
f = open(os.path.join(glove_dir, 'glove.twitter.27B.100d.txt'), encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:], dtype='float32')
    #In dictionary key is word and value is coeff where coeff is vector
    embedding_index[word] = coeff
f.close()

In [29]:
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [30]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [31]:
model_glove = Sequential()
model_glove.add(embedding_layer)
model_glove.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.25))
model_glove.add(Dense(7, activation='softmax'))

model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_glove.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 100)           2613900   
                                                                 
 lstm (LSTM)                 (None, 32)                17024     
                                                                 
 dense (Dense)               (None, 7)                 231       
                                                                 
Total params: 2,631,155
Trainable params: 17,255
Non-trainable params: 2,613,900
_________________________________________________________________
None


In [34]:

checkpoint = ModelCheckpoint(
    '/GloVe/MyDrive/NLP/Word Embeddings/GloVe /Saved_GloVe_model/model-{epoch:03d}-{val_accuracy:03f}.h5', 
     verbose=1, 
     monitor='val_acc',
     save_best_only=True, 
     mode='auto')

In [35]:

history_glove = model_glove.fit(
    X_train_pad, 
    y_train, 
    batch_size=32, 
    epochs=15, 
    validation_data=(X_test_pad, y_test), 
    callbacks=[checkpoint], 
    verbose=1)


Epoch 1/15
2508/2508 [==============================] - ETA: 0s - loss: 0.5741 - accuracy: 0.8122

2508/2508 [==============================] - 157s 62ms/step - loss: 0.5741 - accuracy: 0.8122 - val_loss: 0.4816 - val_accuracy: 0.8408
Epoch 2/15
2508/2508 [==============================] - ETA: 0s - loss: 0.4986 - accuracy: 0.8326

2508/2508 [==============================] - 155s 62ms/step - loss: 0.4986 - accuracy: 0.8326 - val_loss: 0.4409 - val_accuracy: 0.8462
Epoch 3/15
2508/2508 [==============================] - ETA: 0s - loss: 0.4461 - accuracy: 0.8437

2508/2508 [==============================] - 157s 62ms/step - loss: 0.4461 - accuracy: 0.8437 - val_loss: 0.3902 - val_accuracy: 0.8611
Epoch 4/15
2508/2508 [==============================] - ETA: 0s - loss: 0.4079 - accuracy: 0.8545

2508/2508 [==============================] - 161s 64ms/step - loss: 0.4079 - accuracy: 0.8545 - val_loss: 0.3659 - val_accuracy: 0.8649
Epoch 5/15
2508/2508 [==============================] - ETA: 0s - loss: 0.3858 - accuracy: 0.8613

...

2508/2508 [==============================] - 158s 63ms/step - loss: 0.3262 - accuracy: 0.8778 - val_loss: 0.3196 - val_accuracy: 0.8804
Epoch 12/15
2508/2508 [==============================] - ETA: 0s - loss: 0.3210 - accuracy: 0.8798

2508/2508 [==============================] - 158s 63ms/step - loss: 0.3210 - accuracy: 0.8798 - val_loss: 0.3099 - val_accuracy: 0.8837
Epoch 13/15
2508/2508 [==============================] - ETA: 0s - loss: 0.3184 - accuracy: 0.8807

2508/2508 [==============================] - 157s 63ms/step - loss: 0.3184 - accuracy: 0.8807 - val_loss: 0.3055 - val_accuracy: 0.8870
Epoch 14/15
2508/2508 [==============================] - ETA: 0s - loss: 0.3144 - accuracy: 0.8817

2508/2508 [==============================] - 158s 63ms/step - loss: 0.3144 - accuracy: 0.8817 - val_loss: 0.3157 - val_accuracy: 0.8837
Epoch 15/15
2508/2508 [==============================] - ETA: 0s - loss: 0.3126 - accuracy: 0.8845

2508/2508 [==============================] - 158s 63ms/step - loss: 0.3126 - accuracy: 0.8845 - val_loss: 0.3058 - val_accuracy: 0.8850
